In [1]:
import numpy as np
import torch
from torch import nn
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV


In [2]:
# Define the neural network architecture with a variable number of layers
class VariableLayerNet(nn.Module):
    def __init__(self, num_units=10, num_layers=2):
        super(VariableLayerNet, self).__init__()
        layers = [nn.Linear(1, num_units), nn.Sigmoid()]
        
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(num_units, num_units))
            layers.append(nn.Sigmoid())
        
        layers.append(nn.Linear(num_units, 1))  # Output layer
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


In [10]:
num_points = 1000
x_values = np.linspace(0, 4 * np.pi, num_points)
y_values = np.sin(x_values)
y_values[y_values < 0] = 0
y_values[y_values > 0] = 1

In [11]:
x_tensor = torch.tensor(x_values, dtype=torch.float32).view(-1, 1)
y_tensor = torch.tensor(y_values, dtype=torch.float32).view(-1, 1)

In [12]:
num_samples = 900
indices = np.random.choice(range(num_points), num_samples, replace=False)
x_train = x_tensor[indices]
y_train = y_tensor[indices]

In [13]:
params = {
    'module__num_units': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
    'module__num_layers': [2, 3, 4, 6,8,12,15,18 , 20],  # Example: searching the best numlayers
    'max_epochs': [10],  # You can define more epochs
    'lr': [0.01, 0.05, 0.1],  # Learning rates to try
}

In [14]:
# Initialize the neural net regressor
net = NeuralNetRegressor(
    module=VariableLayerNet,
    criterion=nn.MSELoss,
    optimizer=torch.optim.Adam,
    iterator_train__shuffle=True,
)


In [15]:
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='neg_mean_squared_error')



In [16]:
# Run grid search
gs.fit(x_train, y_train)

# Best parameters found
print("Best parameters found:", gs.best_params_)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.7973        1.7100  0.0187
      2        1.6346        1.5525  0.0156
      3        1.4814        1.4089  0.0156
      4        1.3446        1.2774  0.0313
      5        1.2183        1.1568  0.0155
      6        1.1020        1.0452  0.0312
      7        0.9919        0.9419  0.0158
      8        0.8947        0.8449  0.0312
      9        0.8013        0.7557  0.0312
     10        0.7161        0.6742  0.0155
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.4503        0.4197  0.0156
      2        0.3919        0.3680  0.0158
      3        0.3460        0.3275  0.0157
      4        0.3114        0.2975  0.0312
      5        0.2859        0.2764  0.0156
      6        0.2685        0.2626  0.0313
      7        0.2579        0.2542  0.0197
      8        0.2513        0.2496  0.0313
      9        0.2484        0.2

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.2645        0.2551  0.0160
      2        0.2581        0.2541  0.0103
      3        0.2541        0.2494  0.0162
      4        0.2482        0.2485  0.0161
      5        0.2463        0.2467  0.0157
      6        0.2428        0.2437  0.0156
      7        0.2392        0.2411  0.0000
      8        0.2371        0.2392  0.0000
      9        0.2345        0.2378  0.0162
     10        0.2299        0.2360  0.0156
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.5067        0.3884  0.0167
      2        0.3551        0.2903  0.0124
      3        0.2756        0.2546  0.0321
      4        0.2519        0.2562  0.0000
      5        0.2571        0.2657  0.0156
      6        0.2646        0.2671  0.0156
      7        0.2621        0.2601  0.0156
      8        0.2554        0.2512  0.0071
      9        0.2483        0.2

      3        0.2378        0.2395  0.0128
      4        0.2349        0.2369  0.0026
      5        0.2303        0.2356  0.0005
      6        0.2250        0.2325  0.0278
      7        0.2205        0.2292  0.0000
      8        0.2163        0.2269  0.0157
      9        0.2110        0.2279  0.0255
     10        0.2076        0.2241  0.0075
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.5296        0.3604  0.0000
      2        0.3225        0.2481  0.0000
      3        0.2450        0.2394  0.0012
      4        0.2447        0.2593  0.0156
      5        0.2569        0.2563  0.0122
      6        0.2489        0.2371  0.0162
      7        0.2319        0.2226  0.0160
      8        0.2233        0.2174  0.0244
      9        0.2231        0.2157  0.0040
     10        0.2221        0.2120  0.0138
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.2493        0.2

      6        0.2155        0.2291  0.0148
      7        0.2106        0.2247  0.0130
      8        0.2048        0.2202  0.0069
      9        0.2019        0.2205  0.0215
     10        0.1985        0.2209  0.0149
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.2474        0.2356  0.0147
      2        0.2363        0.2292  0.0033
      3        0.2274        0.2151  0.0204
      4        0.2203        0.2059  0.0138
      5        0.2123        0.1991  0.0213
      6        0.2077        0.1916  0.0072
      7        0.2034        0.1869  0.0143
      8        0.2017        0.1843  0.0144
      9        0.2014        0.1829  0.0140
     10        0.2015        0.1819  0.0138
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.3421        0.2557  0.0139
      2        0.2595        0.2876  0.0140
      3        0.2793        0.2702  0.0144
      4        0.2565        0.2

     10        0.2095        0.2237  0.0138
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.7038        0.3232  0.0121
      2        0.2873        0.2994  0.0083
      3        0.3246        0.3469  0.0156
      4        0.3165        0.2722  0.0258
      5        0.2525        0.2349  0.0058
      6        0.2396        0.2424  0.0213
      7        0.2503        0.2370  0.0143
      8        0.2373        0.2176  0.0137
      9        0.2198        0.2123  0.0151
     10        0.2174        0.2136  0.0204
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.2745        0.2678  0.0148
      2        0.2527        0.2359  0.0129
      3        0.2387        0.2276  0.0138
      4        0.2303        0.2196  0.0145
      5        0.2237        0.2164  0.0137
      6        0.2191        0.2023  0.0204
      7        0.2125        0.1946  0.0144
      8        0.2095        0.1

...

      4        0.2628        0.2607  0.0313
      5        0.2598        0.2533  0.0286
      6        0.2567        0.2524  0.0472
      7        0.2544        0.2514  0.0197
      8        0.2506        0.2570  0.0318
      9        0.2559        0.2499  0.0312
     10        0.2527        0.2505  0.0313
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        1.1311        0.8455  0.0156
      2        0.7345        0.3898  0.0312
      3        0.3209        0.2531  0.0281
      4        0.2537        0.2793  0.0312
      5        0.2829        0.2641  0.0325
      6        0.2584        0.2505  0.0397
      7        0.2514        0.2570  0.0471
      8        0.2566        0.2556  0.0388
      9        0.2523        0.2503  0.0483
     10        0.2507        0.2510  0.0307
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.6549        0.3340  0.0466
      2        0.3793        0.2

      8        0.2587        0.2509  0.0156
      9        0.2552        0.2596  0.0313
     10        0.2554        0.2515  0.0312
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.3679        0.3718  0.0313
      2        0.3219        0.2505  0.0312
      3        0.2542        0.2771  0.0156
      4        0.2750        0.2556  0.0469
      5        0.2521        0.2558  0.0272
      6        0.2585        0.2611  0.0174
      7        0.2556        0.2503  0.0367
      8        0.2503        0.2515  0.0325
      9        0.2542        0.2511  0.0312
     10        0.2529        0.2503  0.0156
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.9925        0.2694  0.0322
      2        0.2595        0.3071  0.0321
      3        0.3106        0.3199  0.0312
      4        0.2930        0.2761  0.0354
      5        0.2603        0.2524  0.0314
      6        0.2493        0.2

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.3403        0.2924  0.0313
      2        0.2995        0.2502  0.0312
      3        0.2719        0.2779  0.0157
      4        0.2528        0.2596  0.0331
      5        0.2679        0.2521  0.0391
      6        0.2496        0.2642  0.0274
      7        0.2575        0.2518  0.0403
      8        0.2540        0.2533  0.0398
      9        0.2526        0.2517  0.0343
     10        0.2535        0.2553  0.0440
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.4910        0.2750  0.0269
      2        0.2697        0.3349  0.0326
      3        0.2973        0.2569  0.0313
      4        0.2695        0.2571  0.0227
      5        0.2711        0.2694  0.0156
      6        0.2573        0.2515  0.0312
      7        0.2565        0.2530  0.0313
      8        0.2559        0.2543  0.0328
      9        0.2516        0.2

      2        0.2614        0.2750  0.0262
      3        0.2647        0.2628  0.0255
      4        0.2584        0.2517  0.0312
      5        0.2555        0.2534  0.0323
      6        0.2515        0.2530  0.0323
      7        0.2522        0.2506  0.0312
      8        0.2529        0.2507  0.0312
      9        0.2499        0.2507  0.0312
     10        0.2516        0.2506  0.0156
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.5439        0.2527  0.0338
      2        0.2798        0.3121  0.0260
      3        0.2785        0.2822  0.0312
      4        0.2878        0.2520  0.0290
      5        0.2712        0.2790  0.0156
      6        0.2668        0.2539  0.0312
      7        0.2610        0.2603  0.0312
      8        0.2512        0.2535  0.0312
      9        0.2588        0.2521  0.0312
     10        0.2484        0.2554  0.0297
  epoch    train_loss    valid_loss     dur
-------  ------------  ---------

      8        0.2559        0.2523  0.0395
      9        0.2512        0.2501  0.0315
     10        0.2507        0.2503  0.0407
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.5815        0.3113  0.0324
      2        0.3632        0.2873  0.0297
      3        0.2739        0.2682  0.0268
      4        0.2635        0.2581  0.0271
      5        0.2521        0.2500  0.0312
      6        0.2516        0.2506  0.0293
      7        0.2516        0.2500  0.0313
      8        0.2504        0.2527  0.0313
      9        0.2509        0.2516  0.0320
     10        0.2499        0.2500  0.0312
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.6253        0.5358  0.0312
      2        0.4861        0.2714  0.0303
      3        0.2671        0.2899  0.0450
      4        0.2764        0.2500  0.0322
      5        0.2519        0.2679  0.0398
      6        0.2649        0.2